In [1]:
import json
import numpy as np
import networkx as nx
from scipy.optimize import minimize



### Definimos la función error para el optimizador

In [2]:
def get_Ec(links,w):
    edgelist= [ "%s {'weight':%d}" % (links[i],w[i]) for i in range(len(links)) ]
    G = nx.parse_edgelist(edgelist, nodetype = str)
    return  nx.eigenvector_centrality(G, weight="weight")   



def fobj(pars, args):

    links,players,mu = args

    Ec = get_Ec(links, np.array(pars, dtype=int) ) 

    return sum( (Ec[player]- mu[player])**2 for player in players )


### Preparo los argumentos

In [3]:
# jugadores titulares
players = ['Player1', 'Player2', 'Player3', 'Player4', 'Player5', 'Player6', 'Player7', 'Player8', 'Player9', 'Player10']


# todos los posibles links
links = [ "%s %s"%(players[i],players[j]) for i in range(len(players)) for j in range(i+1, len(players)) ]


#elejimos metricas
arx = 'player_metrica_S1.json'
d = json.load(open(arx, "r"))
metrica = "ec"

mu={  player:np.mean(d[player][metrica]) for player in players }

args = [ links,players,mu  ]


### Optimizo

In [4]:

pars0 = np.ones(len(links))
bounds = [ (0, 10) for i in range(len(links)) ]

res = minimize(fobj, pars0, args=args, method="Powell", bounds=bounds)

res



 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.0067688929958149785
       x: [ 1.922e+00  2.499e+00 ...  1.147e+00  2.365e+00]
     nit: 5
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [-6.505e-01  2.045e-01 ...  9.966e-04  7.173e-03]]
    nfev: 4824

### Chequeo que el resultado sea correcto

In [5]:
pesos = res.x.astype(int)

In [6]:
Ec = get_Ec(links, res.x)

sd={  player:np.std(d[player][metrica]) for player in players }


for player in players:

    print ( mu[player]-sd[player], "| %f |" %Ec[player] , mu[player]+sd[player])


0.18044220211375533 | 0.290898 | 0.3429855492346802
0.07902201176147232 | 0.170427 | 0.251271581483835
0.0807828986223949 | 0.179134 | 0.2338186181540175
0.27375455470403137 | 0.443761 | 0.5997686833680067
0.2924767200784615 | 0.399874 | 0.46023344256027066
0.3422728064770107 | 0.422627 | 0.46005245994202054
0.09132865824080844 | 0.295196 | 0.448860934612693
0.19446521249519094 | 0.321169 | 0.37932076303837564
0.19680952591402529 | 0.330364 | 0.4191910076896765
0.04722332954305739 | 0.139203 | 0.14074474714752194


In [9]:
edgelist= [ "%s %d" % (links[i],pesos[i]) for i in range(len(links)) if pesos[i]>0 ]

np.savetxt("red_optima.edgelist", edgelist, fmt="%s")

